# Tutorial - build MNB with sklearn

This tutorial demonstrates how to use the Sci-kit Learn (sklearn) package to build Multinomial Naive Bayes model, rank features, and use the model for prediction. 

The data from the Kaggle Sentiment Analysis on Movie Review Competition are used in this tutorial. Check out the details of the data and the competition on Kaggle.
https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews

The tutorial also includes sample code to prepare your prediction result for submission to Kaggle. Although the competition is over, you can still submit your prediction to get an evaluation score.

# Step 1: Read in data

In [64]:
# read in the training data

# the data set includes four columns: PhraseId, SentenceId, Phrase, Sentiment
# In this data set a sentence is further split into phrases 
# in order to build a sentiment classification model
# that can not only predict sentiment of sentences but also shorter phrases

# A data example:
# PhraseId SentenceId Phrase Sentiment
# 1 1 A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .1

# the Phrase column includes the training examples
# the Sentiment column includes the training labels
# "0" for very negative
# "1" for negative
# "2" for neutral
# "3" for positive
# "4" for very positive


import pandas as p
train=p.read_csv("../data/training_XL.csv", delimiter=',')
y=train['sentiment'].values
X=train['review'].values

# Step 2: Split train/test data for hold-out test

In [65]:
# check the sklearn documentation for train_test_split
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# "test_size" : float, int, None, optional
# If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. 
# If int, represents the absolute number of test samples. 
# If None, the value is set to the complement of the train size. 
# By default, the value is set to 0.25. The default will change in version 0.21. It will remain 0.25 only if train_size is unspecified, otherwise it will complement the specified train_size.    

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

(61,) (61,) (31,) (31,)
'Ruby Tuesday is my favorite America Style Restaurant. The salad is awesome. And I like the baby pork ribs so much . So does the coconut shrimp.'
p
'After I went shopping with some of my friend we went to DODO restaurant for dinner. I found worm in one of the dishes .'
n


Sample output from the code above:

(93636,) (93636,) (62424,) (62424,)
almost in a class with that of Wilde
3
escape movie
2

# Step 2.1 Data Checking

In [66]:
# Check how many training examples in each category
# this is important to see whether the data set is balanced or skewed

training_labels = set(y_train)
print(training_labels)
from scipy.stats import itemfreq
training_category_dist = itemfreq(y_train)
print(training_category_dist)

{'n', 'p'}
[['n' 30]
 ['p' 31]]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  import sys


The sample output shows that the data set is skewed with 47718/93636=51% "neutral" examples. All other categories are smaller.

{0, 1, 2, 3, 4}
[[    0  4141]
 [    1 16449]
 [    2 47718]
 [    3 19859]
 [    4  5469]]

# Exercise A

In [67]:
# Print out the category distribution in the test data set. 
#Is the test data set's category distribution similar to the training data set's?

# Your code starts here
test_labels = set(y_test)
print(test_labels)
from scipy.stats import itemfreq
test_category_dist = itemfreq(y_test)
print(test_category_dist)

# Your code ends here

{'n', 'p'}
[['n' 16]
 ['p' 15]]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  


# Step 3: Vectorization

In [68]:
# sklearn contains two vectorizers

# CountVectorizer can give you Boolean or TF vectors
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

# TfidfVectorizer can give you TF or TFIDF vectors
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

# Read the sklearn documentation to understand all vectorization options

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# several commonly used vectorizer setting

#  unigram boolean vectorizer, set minimum document frequency to 5
unigram_bool_vectorizer = CountVectorizer(encoding='latin-1', binary=True, min_df=5, stop_words='english')

#  unigram term frequency vectorizer, set minimum document frequency to 5
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=5, stop_words='english')

#  unigram and bigram term frequency vectorizer, set minimum document frequency to 5
gram12_count_vectorizer = CountVectorizer(encoding='latin-1', ngram_range=(1,2), min_df=5, stop_words='english')

#  unigram tfidf vectorizer, set minimum document frequency to 5
unigram_tfidf_vectorizer = TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=5, stop_words='english')


## Step 3.1: Vectorize the training data

In [69]:
# The vectorizer can do "fit" and "transform"
# fit is a process to collect unique tokens into the vocabulary
# transform is a process to convert each document to vector based on the vocabulary
# These two processes can be done together using fit_transform(), or used individually: fit() or transform()

# fit vocabulary in training documents and transform the training documents into vectors
X_train_vec = unigram_count_vectorizer.fit_transform(X_train)

# check the content of a document vector
print(X_train_vec.shape)
print(X_train_vec[0].toarray())

# check the size of the constructed vocabulary
print(len(unigram_count_vectorizer.vocabulary_))

# print out the first 10 items in the vocabulary
print(list(unigram_count_vectorizer.vocabulary_.items())[:10])

# check word index in vocabulary
print(unigram_count_vectorizer.vocabulary_.get('imaginative'))

(61, 62)
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
62
[('favorite', 16), ('restaurant', 42), ('salad', 44), ('like', 28), ('served', 46), ('food', 17), ('service', 47), ('wait', 54), ('minutes', 31), ('menu', 30)]
None


Sample output:

(93636, 11967)
[[0 0 0 ..., 0 0 0]]
11967
[('imaginative', 5224), ('tom', 10809), ('smiling', 9708), ('easy', 3310), ('diversity', 3060), ('impossibly', 5279), ('buy', 1458), ('sentiments', 9305), ('households', 5095), ('deteriorates', 2843)]
5224

## Step 3.2: Vectorize the test data

In [70]:
# use the vocabulary constructed from the training data to vectorize the test data. 
# Therefore, use "transform" only, not "fit_transform", 
# otherwise "fit" would generate a new vocabulary from the test data

X_test_vec = unigram_count_vectorizer.transform(X_test)

# print out #examples and #features in the test set
print(X_test_vec.shape)

(31, 62)


Sample output:

(62424, 14324)

# Exercise B

In [71]:
# In the above sample code, the term-frequency vectors were generated for training and test data.

# Some people argue that 
# because the MultinomialNB algorithm is based on word frequency, 
# we should not use boolean representation for MultinomialNB.
# While in theory it is true, you might see people use boolean representation for MultinomialNB
# especially when the chosen tool, e.g. Weka, does not provide the BernoulliNB algorithm.

# sklearn does provide both MultinomialNB and BernoulliNB algorithms.
# http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html
# You will practice that later

# In this exercise you will vectorize the training and test data using boolean representation
# You can decide on other options like ngrams, stopwords, etc.

# Your code starts here
X_train_vec = unigram_bool_vectorizer.fit_transform(X_train)
print(X_train_vec.shape)
X_test_vec = unigram_bool_vectorizer.transform(X_test)
print(X_test_vec.shape)
# Your code ends here

(61, 62)
(31, 62)


# Step 4: Train a MNB classifier

In [72]:
# import the MNB module
from sklearn.naive_bayes import MultinomialNB

# initialize the MNB model
nb_clf= MultinomialNB()

# use the training data to train the MNB model
nb_clf.fit(X_train_vec,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Step 4.1 Interpret a trained MNB model

In [73]:
## interpreting naive Bayes models
## by consulting the sklearn documentation you can also find out how to print the coef_ for naive Bayes 
## which are the conditional probabilities
## http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

# the code below will print out the conditional prob of the word "worthless" in each category
# sample output
# -8.98942647599 -> logP('worthless'|very negative')
# -11.1864401922 -> logP('worthless'|negative')
# -12.3637684625 -> logP('worthless'|neutral')
# -11.9886066961 -> logP('worthless'|positive')
# -11.0504454621 -> logP('worthless'|very positive')
# the above output means the word feature "worthless" is indicating "very negative" 
# because P('worthless'|very negative) is the greatest among all conditional probs

unigram_count_vectorizer.vocabulary_.get('worthless')
print(nb_clf.feature_log_prob_[0][unigram_count_vectorizer.vocabulary_.get('worthless')])

[[-5.89440283 -3.69717826 -3.94849269 -4.50810847 -4.10264337 -4.50810847
  -4.79579055 -5.20125565 -4.50810847 -4.28496492 -4.50810847 -4.50810847
  -4.28496492 -3.94849269 -4.50810847 -3.69717826 -4.79579055 -2.94996386
  -5.20125565 -5.20125565 -4.28496492 -4.50810847 -3.81496129 -4.79579055
  -4.79579055 -3.94849269 -4.28496492 -4.50810847 -3.81496129 -4.28496492
  -3.94849269 -3.69717826 -5.20125565 -5.20125565 -4.50810847 -4.10264337
  -3.81496129 -4.50810847 -3.49650756 -5.20125565 -4.50810847 -3.94849269
  -2.80336038 -3.81496129 -3.94849269 -5.20125565 -3.94849269 -3.59181774
  -4.79579055 -4.10264337 -4.28496492 -3.81496129 -3.81496129 -3.94849269
  -4.28496492 -4.10264337 -4.50810847 -4.10264337 -4.50810847 -4.28496492
  -3.18635263 -4.79579055]]


Sample output:

-8.5389826392
-10.6436375867
-11.8419845779
-11.4778370023
-10.6297551464

In [74]:
# sort the conditional probability for category 0 "very negative"
# print the words with highest conditional probs
# these can be words popular in the "very negative" category alone, or words popular in all cateogires

feature_ranks = sorted(zip(nb_clf.feature_log_prob_[0], unigram_count_vectorizer.get_feature_names()))
very_negative_features = feature_ranks[-10:]
v_pos_f = feature_ranks[:10]
print(very_negative_features)
print('\n',v_pos_f)

[(-3.8149612925850147, 'terrible'), (-3.8149612925850147, 'time'), (-3.697178256928631, 'asked'), (-3.697178256928631, 'experience'), (-3.697178256928631, 'minutes'), (-3.5918177412708046, 'service'), (-3.49650756146648, 'place'), (-3.1863526331626404, 'went'), (-2.94996385509841, 'food'), (-2.8033603809065344, 'restaurant')]

 [(-5.8944028342648505, 'amazing'), (-5.201255653704905, 'delicious'), (-5.201255653704905, 'fresh'), (-5.201255653704905, 'friendly'), (-5.201255653704905, 'need'), (-5.201255653704905, 'nice'), (-5.201255653704905, 'prices'), (-5.201255653704905, 'sauce'), (-4.7957905455967405, 'definitely'), (-4.7957905455967405, 'favorite')]


In [62]:
# pretty print of top and bottom features

# This is a function I found from stackexchange, and adapted a little bit
# The purpose is to print the top and bottom features nicely
# https://stackoverflow.com/questions/11116697/how-to-get-most-informative-features-for-scikit-learn-classifiers

# You can find many useful scripts from stackexchange or GitHub
# Most tasks are not so unique, so someone in this world might have done something similar and shared their code

def show_most_and_least_informative_features(vectorizer, clf, class_idx=0, n=10):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[class_idx], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[-n:])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [63]:
# show most positive features (category 4)
show_most_and_least_informative_features(unigram_count_vectorizer, nb_clf, class_idx=0, n=10)

	-5.6870	delicious      		-3.7411	service        
	-5.6870	want           		-3.6075	best           
	-4.9938	definitely     		-3.6075	experience     
	-4.9938	dine           		-3.6075	place          
	-4.9938	high           		-3.6075	really         
	-4.9938	prices         		-3.6075	time           
	-4.9938	said           		-3.3844	went           
	-4.9938	sauce          		-3.2021	good           
	-4.9938	staff          		-2.8538	food           
	-4.9938	waiters        		-2.6912	restaurant     


Sample output:

    -10.6298	000            		-5.8176	performance    
	-10.6298	101            		-5.7699	comedy         
	-10.6298	102            		-5.7245	great          
	-10.6298	103            		-5.6953	story          
	-10.6298	104            		-5.6461	performances   
	-10.6298	105            		-5.3723	good           
	-10.6298	10th           		-5.2316	funny          
	-10.6298	110            		-5.1367	best           
	-10.6298	112            		-4.7605	movie          
	-10.6298	12             		-4.2530	film  

In [16]:
nb_clf.coef_[4][0]

-10.625732263919769

# Exercise C

In [38]:
# calculate log ratio of conditional probs

# In this exercise you will calculate the log ratio 
# between conditional probs in the "very negative" category
# and conditional probs in the "very positive" category,
# and then sort and print out the top and bottom 10 words

# the conditional probs for the "very negative" category is stored in nb_clf.coef_[0]
# the conditional probs for the "very positive" category is stored in nb_clf.coef_[4]

# You can consult with similar code in week 4's sample script on feature weighting
# Note that in sklearn's MultinomialNB the conditional probs have been converted to log values.

# Your code starts here
log_ratios = []
features = unigram_count_vectorizer.get_feature_names()
vneg_cond_prob = nb_clf.coef_[0]
vpos_cond_prob = nb_clf.coef_[0]

for i in range(0, len(features)):
  log_ratio = vpos_cond_prob[i] - vneg_cond_prob[i]
  log_ratios.append(log_ratio)

exercise_C_ranks = sorted(zip(log_ratios, features))
print(exercise_C_ranks[:10])
print(exercise_C_ranks[-10:])
    
# Your code ends here

[(0.0, 'amazing'), (0.0, 'asked'), (0.0, 'bad'), (0.0, 'best'), (0.0, 'came'), (0.0, 'chicken'), (0.0, 'definitely'), (0.0, 'delicious'), (0.0, 'didn'), (0.0, 'dine')]
[(0.0, 'time'), (0.0, 'took'), (0.0, 'wait'), (0.0, 'waiter'), (0.0, 'waiters'), (0.0, 'waitress'), (0.0, 'want'), (0.0, 'wasn'), (0.0, 'went'), (0.0, 'worth')]


Sample output for print(log_ratios[0])

-0.838009538739

# Step 5: Test the MNB classifier

In [22]:
# test the classifier on the test data set, print accuracy score

nb_clf.score(X_test_vec,y_test)

0.606401384083045

In [23]:
# print confusion matrix (row: ground truth; col: prediction)

from sklearn.metrics import confusion_matrix
y_pred = nb_clf.fit(X_train_vec, y_train).predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred, labels=[0,1,2,3,4])
print(cm)

[[  733  1264   817   106    11]
 [  602  4132  5411   649    30]
 [  246  2397 25756  3226   239]
 [   19   454  5580  6248   767]
 [    1    54   725  1972   985]]


In [24]:
# print classification report

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
print(precision_score(y_test, y_pred, average=None))
print(recall_score(y_test, y_pred, average=None))

from sklearn.metrics import classification_report
target_names = ['0','1','2','3','4']
print(classification_report(y_test, y_pred, target_names=target_names))

[ 0.45783885  0.49777135  0.67267361  0.51208917  0.48474409]
[ 0.2500853   0.38174427  0.80831032  0.47811448  0.26358041]
             precision    recall  f1-score   support

          0       0.46      0.25      0.32      2931
          1       0.50      0.38      0.43     10824
          2       0.67      0.81      0.73     31864
          3       0.51      0.48      0.49     13068
          4       0.48      0.26      0.34      3737

avg / total       0.59      0.61      0.59     62424



# Step 5.1 Interpret the prediction result

In [25]:
## find the calculated posterior probability
posterior_probs = nb_clf.predict_proba(X_test_vec)

## find the posterior probabilities for the first test example
print(posterior_probs[0])

# find the category prediction for the first test example
y_pred = nb_clf.predict(X_test_vec)
print(y_pred[0])

# check the actual label for the first test example
print(y_test[0])

[ 0.06530109  0.34286038  0.50421851  0.07186049  0.01575954]
2
2


sample output array([ 0.06434628  0.34275846  0.50433091  0.07276319  0.01580115]

Because the posterior probability for category 2 (neutral) is the greatest, 0.50, the prediction should be "2". Because the actual label is also "2", this is a correct prediction


# Step 5.2 Error Analysis

In [26]:
# print out specific type of error for further analysis

# print out the very positive examples that are mistakenly predicted as negative
# according to the confusion matrix, there should be 53 such examples
# note if you use a different vectorizer option, your result might be different

err_cnt = 0
for i in range(0, len(y_test)):
    if(y_test[i]==4 and y_pred[i]==1):
        print(X_test[i])
        err_cnt = err_cnt+1
print("errors:", err_cnt)

this might not seem like the proper cup of tea , however it is almost guaranteed that even the stuffiest cinema goers will laugh their \*\*\* off for an hour-and-a-half
Parents may even find that it goes by quickly , because it has some of the funniest jokes of any movie this year , including those intended for adults .
of the best short story writing
are an absolute joy .
Adams , with four scriptwriters , takes care with the characters , who are so believable that you feel what they feel .
this might not seem like the proper cup of tea , however it is almost guaranteed that even the stuffiest cinema goers will laugh their \*\*\* off for an hour-and-a-half .
you will probably like it .
the funniest jokes
the film is never dull
, Lawrence 's delivery remains perfect
one of the most high-concept sci fi adventures attempted for the screen
We 've seen the hippie-turned-yuppie plot before , but there 's an enthusiastic charm in Fire that makes the formula fresh again
that it goes by quickly

# Exercise D

In [27]:
# Can you find linguistic patterns in the above errors? 
# What kind of very positive examples were mistakenly predicted as negative?

# Can you write code to print out the errors that very negative examples were mistakenly predicted as very positive?
# Can you find lingustic patterns for this kind of errors?
# Based on the above error analysis, what suggestions would you give to improve the current model?

# Your code starts here
err_cnt = 0
for i in range(0, len(y_test)):
    if(y_test[i]==0 and y_pred[i]==4):
        print(X_test[i])
        err_cnt = err_cnt+1
print("errors:", err_cnt)
# Your code ends here

this is the opposite of a truly magical movie .
achieves the remarkable feat of squandering a topnotch foursome of actors
a deeply unpleasant experience
hugely overwritten
is not Edward Burns ' best film
Once the expectation of laughter has been quashed by whatever obscenity is at hand , even the funniest idea is n't funny .
is a deeply unpleasant experience .
is hugely overwritten ,
is the opposite of a truly magical movie .
to this shocking testament to anti-Semitism and neo-fascism
is about as humorous as watching your favorite pet get buried alive
errors: 11


# Step 6: write the prediction output to file

In [28]:
y_pred=nb_clf.predict(X_test_vec)
output = open('/Users/byu/Desktop/data/prediction_output.csv', 'w')
for x, value in enumerate(y_pred):
  output.write(str(value) + '\n') 
output.close()

# Step 6.1 Prepare submission to Kaggle sentiment classification competition

In [29]:
########## submit to Kaggle submission

# we are still using the model trained on 60% of the training data
# you can re-train the model on the entire data set 
#   and use the new model to predict the Kaggle test data
# below is sample code for using a trained model to predict Kaggle test data 
#    and format the prediction output for Kaggle submission

# read in the test data
kaggle_test=p.read_csv("/Users/byu/Desktop/data/kaggle/test.tsv", delimiter='\t') 

# preserve the id column of the test examples
kaggle_ids=kaggle_test['PhraseId'].values

# read in the text content of the examples
kaggle_X_test=kaggle_test['Phrase'].values

# vectorize the test examples using the vocabulary fitted from the 60% training data
kaggle_X_test_vec=unigram_count_vectorizer.transform(kaggle_X_test)

# predict using the NB classifier that we built
kaggle_pred=nb_clf.fit(X_train_vec, y_train).predict(kaggle_X_test_vec)

# combine the test example ids with their predictions
kaggle_submission=zip(kaggle_ids, kaggle_pred)

# prepare output file
outf=open('/Users/byu/Desktop/data/kaggle/kaggle_submission.csv', 'w')

# write header
outf.write('PhraseId,Sentiment\n')

# write predictions with ids to the output file
for x, value in enumerate(kaggle_submission): outf.write(str(value[0]) + ',' + str(value[1]) + '\n')

# close the output file
outf.close()

# Exercise E

In [ ]:
# generate your Kaggle submissions with boolean representation and TF representation
# submit to Kaggle
# report your scores here
# which model gave better performance in the hold-out test
# which model gave better performance in the Kaggle test

Sample output:

(93636, 9968)
[[0 0 0 ..., 0 0 0]]
9968
[('disloc', 2484), ('surgeon', 8554), ('camaraderi', 1341), ('sketchiest', 7943), ('dedic', 2244), ('impud', 4376), ('adopt', 245), ('worker', 9850), ('buy', 1298), ('systemat', 8623)]
245

# BernoulliNB

In [82]:
from sklearn.naive_bayes import BernoulliNB
X_train_vec_bool = unigram_bool_vectorizer.fit_transform(X_train)
bernoulliNB_clf = BernoulliNB(X_train_vec_bool, y_train)


# Cross Validation

In [40]:
# cross validation

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
nb_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=False)),('nb', MultinomialNB())])
scores = cross_val_score(nb_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)

0.8256944444444444


# Exercise F

In [76]:
# run 3-fold cross validation to compare the performance of 
# (1) BernoulliNB (2) MultinomialNB with TF vectors (3) MultinomialNB with boolean vectors

# Your code starts here

##Bernoulli
bNB_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1',binary=False)),('bernNB',BernoulliNB(fit_prior=True))])
scores = cross_val_score(bNB_pipe,X,y,cv=3)
print(sum(scores)/len(scores))

##MNB TFIDF
mNB_tfidf_pipe = Pipeline([('nb_tf',TfidfVectorizer(encoding='latin-1',use_idf=True,binary=False)),('nb',MultinomialNB())])
scores = cross_val_score(mNB_tfidf_pipe,X,y,cv=3)
print(sum(scores)/len(scores))

##MNB TF
mNB_tf_pipe = Pipeline([('nb_tf',TfidfVectorizer(encoding='latin-1',use_idf=False,binary=False)),('nb',MultinomialNB())])
scores = cross_val_score(mNB_tf_pipe,X,y,cv=3)
print(sum(scores)/len(scores))

##MNB with Bool
nb_clf_pipe = Pipeline([('vect', CountVectorizer(encoding='latin-1', binary=True)),('nb', MultinomialNB())])
scores = cross_val_score(nb_clf_pipe, X, y, cv=3)
avg=sum(scores)/len(scores)
print(avg)
# Your code ends here


0.7604166666666666
0.8375
0.7937500000000001
0.8263888888888888


AttributeError: 'Pipeline' object has no attribute 'feature_log_prob_'

# Optional: use external linguistic resources such as stemmer

In [204]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk.stem

english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([english_stemmer.stem(w) for w in analyzer(doc)])

stem_vectorizer = StemmedCountVectorizer(min_df=3, analyzer="word")
X_train_stem_vec = stem_vectorizer.fit_transform(X_train)

In [194]:
# check the content of a document vector
print(X_train_stem_vec.shape)
print(X_train_stem_vec[0].toarray())

# check the size of the constructed vocabulary
print(len(stem_vectorizer.vocabulary_))

# print out the first 10 items in the vocabulary
print(list(stem_vectorizer.vocabulary_.items())[:10])

# check word index in vocabulary
print(stem_vectorizer.vocabulary_.get('adopt'))

(93636, 9968)
[[0 0 0 ..., 0 0 0]]
9968
[('disloc', 2484), ('surgeon', 8554), ('camaraderi', 1341), ('sketchiest', 7943), ('dedic', 2244), ('impud', 4376), ('adopt', 245), ('worker', 9850), ('buy', 1298), ('systemat', 8623)]
245
